In [2]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as rp
import gmaps
from config import gkey

In [22]:
base = "http://ufo-api.herokuapp.com/api/sightings/location/near?"
Ontario_lat_max = 56.800878
Ontario_lat_min = 41.771312
Ontario_lng_max = -75.251467
Ontario_lng_min = -95.281572

lats = np.random.uniform(low=Ontario_lat_min, high=Ontario_lat_max, size=500)
lngs = np.random.uniform(low=Ontario_lng_min, high=Ontario_lng_max, size=500)


In [23]:
len(lats), len(lngs)

(500, 500)

In [24]:
city = []
shape = []
state = []
date = []
duration = []
lats_ON = []
lngs_ON = []
for i in np.arange(0,len(lats)):
    try:
        responce = rp.get(f"{base}lat={lats[i]}&lon={lngs[i]}&limit=1").json()
        if responce["sightings"][0]["obj"]["state"] == "ON":
            city.append(responce["sightings"][0]["obj"]["city"])
            shape.append(responce["sightings"][0]["obj"]["shape"])
            state.append(responce["sightings"][0]["obj"]["state"])
            date.append(responce["sightings"][0]["obj"]["date"])
            duration.append(responce["sightings"][0]["obj"]["duration"])
            lats_ON.append(responce["sightings"][0]["obj"]["loc"][1])
            lngs_ON.append(responce["sightings"][0]["obj"]["loc"][0])
    except Exception as e:
        print("could not find")
print("finished")

finished


In [25]:
Combined = pd.DataFrame({"lat":lats_ON,"lng":lngs_ON,"city":city,"shape":shape,"state":state,"date":date,"duration":duration})
Combined

,lat,lng,city,shape,state,date,duration
0,49.105598,-81.013603,Cochrane (Canada),Unknown,ON,2012-09-04T22:00:00.000Z,split second
1,52.923989,-82.428349,Attawapiskat (Canada),Sphere,ON,2012-10-08T21:45:00.000Z,5 minutes
2,51.266667,-80.650000,Moosonee (Canada),Flash,ON,2007-01-18T17:30:00.000Z,
3,44.260030,-76.505992,Kingston (Canada),Circle,ON,2015-03-12T21:30:00.000Z,40 minutes
4,46.449094,-82.690503,Elliot Lake (Canada),Unknown,ON,2016-05-23T23:30:00.000Z,5 minutes
5,47.820932,-83.354986,Chapleau (Canada),Light,ON,2009-09-23T22:00:00.000Z,1 hour
6,52.923989,-82.428349,Attawapiskat (Canada),Sphere,ON,2012-10-08T21:45:00.000Z,5 minutes
7,52.923989,-82.428349,Attawapiskat (Canada),Sphere,ON,2012-10-08T21:45:00.000Z,5 minutes
8,50.097436,-91.916516,Sioux Lookout (Canada),Oval,ON,2009-03-07T23:00:00.000Z,60 seconds
9,42.531399,-81.606903,Rodney (Canada),Light,ON,2002-09-06T22:50:00.000Z,4 minutes


In [53]:
gkey = ""
gmaps.configure(api_key=gkey)
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [59]:

coordinates = Combined[["lat","lng"]]
coordinates
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [60]:
heat_layer = gmaps.heatmap_layer(coordinates, weights=np.arange(0,len(lats_ON)), 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1.0)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius = 1

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [ ]:
# name = []
# price = []

# for i in np.arange(0,len(x)):
#     name.append(x[i]["name"])
#     price.append(x[i]["price"])

In [ ]:
# y = pd.DataFrame({"Name":name, "Price":price})
# #
# y.sort_values("Price",)